In [61]:
import pandas as ps
import numpy as np
from grid import location_to_cell_id
from sqlalchemy import create_engine
from math import ceil

engine = create_engine('sqlite://', echo=False)

## 0. Read Dataframes

In [2]:
pois_df = ps.read_csv('pois.csv', sep=' ')
queries_df = ps.read_csv('queries.csv', sep=' ')

print(pois_df)
print(queries_df)

      poi_id  cell_id     poi_type        lat       lon
0        307        1          bar  46.504486  6.559631
1        331        1          bar  46.500259  6.554721
2        578        1  supermarket  46.506017  6.551165
3        897        1         club  46.504494  6.553889
4        972        1  supermarket  46.502984  6.550519
...      ...      ...          ...        ...       ...
1055     372      100   restaurant  46.563852  6.648454
1056     649      100          bar  46.564047  6.640250
1057     730      100          gym  46.565961  6.642228
1058     828      100         club  46.569177  6.642401
1059     990      100          bar  46.565290  6.640855

[1060 rows x 5 columns]
           ip_address        lat       lon   timestamp poi_type_query
0      34.101.177.245  46.532942  6.591174   14.912448      cafeteria
1      34.101.177.245  46.532942  6.591174   14.912448     restaurant
2      34.101.177.245  46.550342  6.602852   18.024657     restaurant
3      34.101.177.245  

## 1. Data cleaning

In [3]:
#Extract cell_id from which query happened
def get_cell_id(row):
    return location_to_cell_id(row['lat'], row['lon'])

queries_df['cell_id'] = queries_df.apply(lambda row: get_cell_id(row), axis=1)
print(queries_df)

           ip_address        lat       lon   timestamp poi_type_query  cell_id
0      34.101.177.245  46.532942  6.591174   14.912448      cafeteria       45
1      34.101.177.245  46.532942  6.591174   14.912448     restaurant       45
2      34.101.177.245  46.550342  6.602852   18.024657     restaurant       76
3      34.101.177.245  46.550342  6.602852   18.024657      cafeteria       76
4      34.101.177.245  46.532942  6.591174   36.334539      cafeteria       45
...               ...        ...       ...         ...            ...      ...
20438     11.173.13.2  46.524410  6.625246  449.159554    supermarket       38
20439     11.173.13.2  46.527363  6.628705  453.426750    supermarket       38
20440     11.173.13.2  46.527363  6.628705  453.426750            gym       38
20441     11.173.13.2  46.524410  6.625246  464.420041    supermarket       38
20442     11.173.13.2  46.527363  6.628705  464.420041     restaurant       38

[20443 rows x 6 columns]


In [29]:
#Get day of the query and time
def get_day(row):
    return ceil(row['timestamp'] / 24)

def get_hour_of_day(row):
    return int(row['timestamp'] % 24)

queries_df['day'] = queries_df.apply(lambda row: get_day(row), axis=1)
queries_df['time'] = queries_df.apply(lambda row: get_hour_of_day(row), axis=1)

#Get daytime
def get_daytime(row):
    time = row['time']
    if (time >= 0 and time < 9):
        return '1.Early'
    if (time >= 9 and time < 12):
        return '2.Morning'
    if (time >= 12 and time < 17):
        return '3.Afternoon'
    if (time >= 17 and time < 20):
        return '4.Evening'
    if (time >= 20 and time <= 23):
        return '5.Night'
queries_df['daytime'] = queries_df.apply(lambda row: get_daytime(row), axis=1)

print(queries_df)

           ip_address        lat       lon   timestamp poi_type_query  \
0      34.101.177.245  46.532942  6.591174   14.912448      cafeteria   
1      34.101.177.245  46.532942  6.591174   14.912448     restaurant   
2      34.101.177.245  46.550342  6.602852   18.024657     restaurant   
3      34.101.177.245  46.550342  6.602852   18.024657      cafeteria   
4      34.101.177.245  46.532942  6.591174   36.334539      cafeteria   
...               ...        ...       ...         ...            ...   
20438     11.173.13.2  46.524410  6.625246  449.159554    supermarket   
20439     11.173.13.2  46.527363  6.628705  453.426750    supermarket   
20440     11.173.13.2  46.527363  6.628705  453.426750            gym   
20441     11.173.13.2  46.524410  6.625246  464.420041    supermarket   
20442     11.173.13.2  46.527363  6.628705  464.420041     restaurant   

       cell_id  day  time      daytime  
0           45    1    14  3.Afternoon  
1           45    1    14  3.Afternoon  


## 2.Statistics

In [53]:
!rm stats1.csv
queries_grouped = queries_df.groupby(['ip_address', 'daytime', 'day','cell_id'])\
    .size().sort_index(level=[0,2,1])
#queries_grouped = queries_grouped.where(queries_grouped['count'] > 4).dropna()

queries_grouped = queries_grouped.to_frame(name='count').reset_index()
print(queries_grouped)
queries_grouped.to_csv('stats1.csv')

         ip_address      daytime  day  cell_id  count
0       0.98.248.97    2.Morning    1       63      2
1       0.98.248.97    4.Evening    1       18      2
2       0.98.248.97    2.Morning    2       63      2
3       0.98.248.97      5.Night    2       18      4
4       0.98.248.97    2.Morning    3       63      2
...             ...          ...  ...      ...    ...
9672  97.138.146.97  3.Afternoon   19       22      2
9673  97.138.146.97  3.Afternoon   19       53      2
9674  97.138.146.97    2.Morning   20       10      2
9675  97.138.146.97    2.Morning   20       22      1
9676  97.138.146.97    2.Morning   20       40      1

[9677 rows x 5 columns]


#### queries_grouped contains, for each ip, for each day and daytime,
#### the number of queries launched from a certain cell_id
#### it could be useful for inferring movement patterns

In [ ]:
!rm stats2.csv
queries_grouped_day_removed = queries_grouped.drop(['day'],axis = 1)
max_cell_daytime = queries_grouped_day_removed.groupby(['ip_address','daytime'])\
    .agg(['max'])
max_cell_daytime.to_csv('stats2.csv')

#### max_cell_daytime contains, for each ip, for each daytime, the cell
#### from which most queries have been made
#### it could be useful to individuate home/work cell id in grid

In [102]:
#reset the multi index manually
#i.e just removed first two rows from stats2 and created header
max_cell_daytime = ps.read_csv('stats2.csv', sep=',', header='infer')

In [116]:
class Filter:
    def __init__(self, max_cell_daytime):
        self.max_cell_daytime = max_cell_daytime

    def filter_locations(self, user, daytime, cell_id):
        max_cell_daytime = self.max_cell_daytime
        cell = max_cell_daytime.loc[(max_cell_daytime['ip_address'] == user) & (max_cell_daytime['daytime'] == daytime)]['cell_id'].to_numpy()[0]
        if cell_id != cell:
            return False
        else:
            return True
custom_filter = Filter(max_cell_daytime)
queries_df['filter'] = np.vectorize(custom_filter.filter_locations)(queries_df['ip_address'], queries_df['daytime'], queries_df['cell_id'])
queries_df = queries_df[queries_df['filter'] == True].drop(columns=['filter'])
queries_df = queries_df.sort_values(by=['ip_address', 'day', 'time', 'daytime'], axis=0)
queries_df.to_csv('queries_filtered.csv')

#### filtering queries with locations from cell_id which
#### turned out to be the most interesting cells